In [1]:
import pandas as pd
import numpy as np
import re
import os

# Scrapping Data

In [2]:
def scrapper(fname, basepath = "../Fixed-Fixed_Reports/"):
    with open(basepath+fname, "r") as file:
        data = file.read()
        data = data.replace("\x00", "")[2:]
    file.close()

    pattern = re.findall(r"TABLE 11.+<td>.+</td>.+TABLE 12", data)[0]
    freq = list(map(lambda x: float(x.split("<td>")[-1]), re.findall(r"[0-9]+.</td><td>[0-9]+.[0-9]*", pattern)))

    pattern = re.findall(r"TABLE 13.+<td>.+</td>.+TABLE 14", data)[0]
    pattern = re.findall(r'Maximum</th><td>.+Average', pattern)[0]
    maximum = list(map(lambda x: float(x), re.findall(r"[0-9]+.[0-9]*", pattern)))

    pattern = re.findall(r"TABLE 13.+<td>.+</td>.+TABLE 14", data)[0]
    pattern = re.findall(r'Average</th><td>.+Minimum', pattern)[0]
    average = list(map(lambda x: float(x), re.findall(r"[0-9]+.[0-9]*", pattern)))

    vals  = np.array(freq + maximum + average).reshape(3, 6).T
    vals = np.hstack((np.ones((6, 1))*int(fname.split(".")[0]), np.hstack((np.arange(1, 7).reshape(6, 1), vals))))
    
    return vals

In [3]:
fnames = list(os.walk("../"))[3][2][:-2]

In [4]:
data = scrapper(fnames[0])

In [5]:
for x in fnames[1:]:
    data = np.vstack((data, scrapper(x)))

# Saving as Dataframe

In [6]:
colnames = ["Index", "Mode", "Frequency", "Max_Deflection", "Average_Deflection"]

In [7]:
df = pd.DataFrame(data, columns=colnames)

In [8]:
df = df.sort_values("Index")

# Exporting File

In [9]:
df.to_csv("Fixed-Fixed.csv", index=False)